## Reconcile Products Table
#### Contributors:
##### Tean 8: Anthony Ung, Sean Jerzewski, Gideon Kipkorir

## 0. Dependencies and Global Variables

In [1]:
import os
from enum import Enum
import csv
import sqlite3 as lite
from decimal import Decimal
from datetime import date, datetime, timedelta

In [2]:
DB_HANDLES = {}

## 1. Gather the file paths
  
  
## IMPORTANT: 
#### Most of these files are untracked on GitHub. it is each team members'   
####   &emsp; &emsp; It is each team members' individual responsibilities  
####   &emsp; &emsp; to build the Database and CSV files for themselves using the other Jupyter notebooks. 

In [3]:
FILE_PATHS = {
    'DATA_MART' : './../01_Source_DBs/Region_C_Data_Mart.db'
}

DB_HANDLES = {}

In [4]:
ALL_FILES_OK = True

for file_key in FILE_PATHS:
    file_name = FILE_PATHS[file_key]
    file_exists = os.path.isfile(file_name)
    
    if(file_exists):
        print(f'OK - {file_key} - \'{file_name}\'')
    else:
        ALL_FILES_OK = False
        print(f'MISSING - {file_key} - \'{file_name}\'')

if not ALL_FILES_OK:
    raise SystemExit('\n' "ERROR!" '\n' "You are missing files!" '\n' "Read and Follow the Cell instructions provided.")

OK - DATA_MART - './../01_Source_DBs/Region_C_Data_Mart.db'


## 2. Initialize the Database File and the Database API

I originally made this Database API back in HW 2 and developed it throughout the semester

In [5]:
'''
    If I try to make db_options an inner class to db, 
        I get an error saying that the class is undefined.
'''
class db_options(Enum):
        DEFAULT = 0
        RETURN_RESULTS = 1
        PRINT_RESULTS = 2

class db:
    
    def __init__(self, name):
        self.name = rf"{name}"

    def connect(self):
        self.con = lite.connect(self.name)
        self.cur = self.con.cursor()

    def build_table(self, name):      
        self.execute_sql(f'DROP TABLE IF EXISTS {name}')
        self.execute_sql(TABLE_DEFINITIONS[name])
    
    def execute_sql(self, sql, options=db_options.DEFAULT):
        if (options.value & db_options.RETURN_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            return results
        elif (options.value & db_options.PRINT_RESULTS.value):
            results = self.cur.execute(sql).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql)

    def execute_sql_values(self, sql, values, options=db_options.DEFAULT):
        if (options.value & db_options.RETURN_RESULTS.value):
            results = self.cur.execute(sql, values).fetchall()
            return results
        elif (options.value & db_options.PRINT_RESULTS.value):
            results = self.cur.execute(sql, values).fetchall()
            for row in results:
                print(row)
        else:
            self.cur.execute(sql, values)


    def commit(self):
        self.con.commit()

    def close(self):
        self.con.commit()
        self.con.close()

In [6]:
DB_HANDLES['DATA_MART'] = db(FILE_PATHS['DATA_MART'])

## 3. Top Selling Items for 2024 Q1 and 2025 Q1

#### Known Bug
ETL Script displays 2024 Q1 instead of 2025 Q1

In [16]:
def run():
    db_handle = DB_HANDLES['DATA_MART']

    sql = \
        '''
            SELECT *,
                RANK() OVER (ORDER BY iq.TotalSoldByStoreThisQuarter DESC) AS rank
            FROM inventory_quarterly AS iq
            JOIN products AS p
            WHERE (iq.Quarter = 1) AND (iq.Year = 2025)
            ORDER BY rank DESC
            LIMIT 1
        '''

    db_handle.connect()
    results = db_handle.execute_sql(sql, options=db_options.RETURN_RESULTS)
    for row in results:
        print(row)
    db_handle.close()
    
run()

(1294, 8, 'Q1 2024', 1, 2025, 10, 10, 2, 358.8, 90.0, 358.8, 90.0, 1, '42081001', 'Jambalaya Rice Mix', '57', 'Rice', 'Starchy Foods', 'Starchy Foods', 'Food', '12 oz', 'Zatarains', 'Rowan Warehouse', 4303551)
